In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE182798"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE182798"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE182798.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE182798.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE182798.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of adult-onset asthma related to damp and moldy buildings and idiopathic environmental intolerance"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['diagnosis: adult-onset asthma', 'diagnosis: IEI', 'diagnosis: healthy'], 1: ['gender: Female'], 2: ['age: 33.42', 'age: 46.08', 'age: 45.58', 'age: 28', 'age: 25.75', 'age: 59.83', 'age: 41.17', 'age: 47.58', 'age: 50.75', 'age: 42.58', 'age: 52.75', 'age: 51.75', 'age: 18.42', 'age: 47', 'age: 38.33', 'age: 58.58', 'age: 56.17', 'age: 40.67', 'age: 47.5', 'age: 54.67', 'age: 48.83', 'age: 64.67', 'age: 54.83', 'age: 57.67', 'age: 39.17', 'age: 38.08', 'age: 28.42', 'age: 40.75', 'age: 43.17', 'age: 43.08'], 3: ['cell type: PBMC', 'tissue: Nasal biopsy'], 4: [nan, 'subject: 605', 'subject: 611', 'subject: 621', 'subject: 35', 'subject: 11', 'subject: 1', 's

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional, Callable, Any, Dict

# 1. Gene Expression Data Availability
# Based on the background information, this dataset seems to contain transcriptomic profiling
# which suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify keys for trait, age, and gender
trait_row = 0  # diagnosis information is in row 0
age_row = 2    # age information is in row 2
gender_row = 1 # gender information is in row 1

# 2.2 Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary (0 for control, 1 for Asthma)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if "adult-onset asthma" in value.lower():
        return 1  # Asthma
    elif "healthy" in value.lower():
        return 0  # Control
    else:
        return None  # IEI or other conditions

def convert_age(value: str) -> Optional[float]:
    """Convert age values to float."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Initial filtering validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Assuming clinical_data was loaded in a previous step
        # If not, we need to load it
        if 'clinical_data' not in locals() and 'clinical_data' not in globals():
            # Define the expected path for clinical data
            clinical_data_path = os.path.join(in_cohort_dir, f"{cohort}_sample_characteristics.csv")
            if os.path.exists(clinical_data_path):
                clinical_data = pd.read_csv(clinical_data_path)
            else:
                print(f"Clinical data file not found at {clinical_data_path}")
                # Consider alternative loading methods if needed
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the processed data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
else:
    print("Clinical data not available, skipping clinical feature extraction.")


Preview of selected clinical features:
{'GSM5530417': [1.0, 33.42, 0.0], 'GSM5530418': [1.0, 46.08, 0.0], 'GSM5530419': [nan, 45.58, 0.0], 'GSM5530420': [0.0, 28.0, 0.0], 'GSM5530421': [0.0, 25.75, 0.0], 'GSM5530422': [1.0, 59.83, 0.0], 'GSM5530423': [0.0, 41.17, 0.0], 'GSM5530424': [1.0, 47.58, 0.0], 'GSM5530425': [1.0, 50.75, 0.0], 'GSM5530426': [0.0, 42.58, 0.0], 'GSM5530427': [1.0, 52.75, 0.0], 'GSM5530428': [1.0, 51.75, 0.0], 'GSM5530429': [0.0, 18.42, 0.0], 'GSM5530430': [0.0, 47.0, 0.0], 'GSM5530431': [0.0, 38.33, 0.0], 'GSM5530432': [1.0, 58.58, 0.0], 'GSM5530433': [1.0, 56.17, 0.0], 'GSM5530434': [1.0, 52.75, 0.0], 'GSM5530435': [nan, 40.67, 0.0], 'GSM5530436': [1.0, 47.5, 0.0], 'GSM5530437': [1.0, 54.67, 0.0], 'GSM5530438': [0.0, 48.83, 0.0], 'GSM5530439': [0.0, 25.75, 0.0], 'GSM5530440': [1.0, 64.67, 0.0], 'GSM5530441': [1.0, 54.83, 0.0], 'GSM5530442': [nan, 57.67, 0.0], 'GSM5530443': [nan, 39.17, 0.0], 'GSM5530444': [0.0, 38.08, 0.0], 'GSM5530445': [1.0, 28.42, 0.0], 'GSM55

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE182798/GSE182798_series_matrix.txt.gz


Gene data shape: (39341, 182)
First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315529', 'A_19_P00315543',
       'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315584', 'A_19_P00315593',
       'A_19_P00315603', 'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641',
       'A_19_P00315647', 'A_19_P00315649', 'A_19_P00315668', 'A_19_P00315691'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data do not appear to be standard human gene symbols.
# They appear to be Agilent microarray probe IDs (starting with "A_19_P"), which are platform-specific
# identifiers that need to be mapped to standard gene symbols.
# These probe IDs typically need mapping to official gene symbols for downstream analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# From inspecting previous outputs, I can see:
# - Gene expression data has identifiers like 'A_19_P00315452'
# - Gene annotation data has 'ID' column with values like 'A_21_P0014386', 'A_33_P3396872', etc.
# - The gene symbols are in the 'GENE_SYMBOL' column

# 1. Determine which columns to use for mapping
# The ID column in gene_annotation contains microarray probe IDs similar to gene expression indices
# The GENE_SYMBOL column contains the human gene symbols we want to map to
id_column = 'ID'
gene_symbol_column = 'GENE_SYMBOL'

# 2. Get gene mapping dataframe by extracting relevant columns
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (48862, 2)
Sample of gene mapping:
{'ID': ['A_33_P3396872', 'A_33_P3267760', 'A_32_P194264', 'A_23_P153745', 'A_21_P0014180'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}


Gene expression data shape after mapping: (22196, 182)
First few gene symbols after mapping:
['A1BG', 'A1BG-AS1', 'A1CF-3', 'A2M', 'A2M-1', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'AAAS']


Gene expression data saved to ../../output/preprocess/Asthma/gene_data/GSE182798.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define the correct convert_trait function as established in Step 2
def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary (0 for control, 1 for Asthma)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if "adult-onset asthma" in value.lower():
        return 1  # Asthma
    elif "healthy" in value.lower():
        return 0  # Control
    else:
        return None  # IEI or other conditions

def convert_age(value: str) -> Optional[float]:
    """Convert age values to float."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# Re-extract clinical features using the appropriate conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # Correct trait row from Step 2
    convert_trait=convert_trait,
    age_row=2,     # Age row from Step 2
    convert_age=convert_age,
    gender_row=1,  # Gender row from Step 2
    convert_gender=convert_gender
)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from adult patients with asthma related to damp/moldy buildings and controls."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE182798.csv
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE182798.csv
Linked data shape: (182, 18157)
Linked data preview (first 5 rows, 5 columns):
            Asthma    Age  Gender     A1BG  A1BG-AS1
GSM5530417     1.0  33.42     0.0  9.58489   6.85151
GSM5530418     1.0  46.08     0.0  9.65673   6.74284
GSM5530419     NaN  45.58     0.0  9.55825   6.47113
GSM5530420     0.0  28.00     0.0  9.71055   6.90045
GSM5530421     0.0  25.75     0.0  9.22204   6.75952


Data shape after handling missing values: (152, 18157)
For the feature 'Asthma', the least common label is '0.0' with 57 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.33
  50% (Median): 47.0
  75%: 52.75
Min: 18.42
Max: 64.67
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 152 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Asthma/GSE182798.csv
